In [ ]:
import interpreTS as it
import pandas as pd
import numpy as np
import time
from tsfresh import extract_features
it.__version__

In [11]:
data = pd.DataFrame({
    'id': np.repeat(range(100), 100),
    'time': np.tile(range(100), 100),
    'value1': np.tile(range(100), 100),
    'value2': np.random.normal(0, 1, 10000),
    'value3': np.random.normal(20, 10, 10000)
})
data.head()

,id,time,value1,value2,value3
0,0,0,0,-0.412961,19.934019
1,0,1,1,0.592291,16.939696
2,0,2,2,0.634733,15.915280
3,0,3,3,-0.221277,24.863888
4,0,4,4,-0.112471,24.779529


In [12]:
DEFAULT_FEATURES_IT = [
    it.Features.CROSSING_POINTS,
    it.Features.MEAN,
    it.Features.VARIANCE,
    it.Features.PEAK,
    it.Features.TROUGH,
]

In [13]:
custom_fc_parameters = {
    'number_crossing_m': [{'m': 0}],             # Crossing points at m=0
    'mean': None,
    'variance': None,
    'maximum': None,
    'minimum': None,
}

In [14]:
extractor_it = it.FeatureExtractor(
    window_size=100, 
    stride=100, 
    features=DEFAULT_FEATURES_IT,
    id_column='time',
    sort_column='id'
)

In [15]:
start = time.time()
features_it = extractor_it.extract_features(data)
end = time.time()

features_it.head()

,crossing_points_value1,crossing_points_value2,crossing_points_value3,mean_value1,mean_value2,mean_value3,variance_value1,variance_value2,variance_value3,peak_value1,peak_value2,peak_value3,trough_value1,trough_value2,trough_value3
0,"{'crossing_count': 2, 'crossing_points': [0, 50]}","{'crossing_count': 54, 'crossing_points': [0, ...","{'crossing_count': 47, 'crossing_points': [1, ...",49.5,0.179888,20.243567,841.666667,1.088506,96.054702,99,3.186941,42.527250,0,-2.245841,-8.294389
1,"{'crossing_count': 1, 'crossing_points': [49]}","{'crossing_count': 49, 'crossing_points': [0, ...","{'crossing_count': 53, 'crossing_points': [0, ...",49.5,-0.003050,21.497116,841.666667,1.029674,103.399795,99,2.206501,51.297614,0,-2.568951,-0.267103
2,"{'crossing_count': 1, 'crossing_points': [49]}","{'crossing_count': 47, 'crossing_points': [1, ...","{'crossing_count': 52, 'crossing_points': [0, ...",49.5,-0.016470,21.365738,841.666667,1.094658,102.065842,99,1.887334,46.152546,0,-2.979081,-17.886584
3,"{'crossing_count': 1, 'crossing_points': [49]}","{'crossing_count': 51, 'crossing_points': [0, ...","{'crossing_count': 45, 'crossing_points': [0, ...",49.5,-0.071130,20.507009,841.666667,0.889199,90.446352,99,2.262206,46.448314,0,-2.717986,-8.567784
4,"{'crossing_count': 1, 'crossing_points': [49]}","{'crossing_count': 48, 'crossing_points': [1, ...","{'crossing_count': 56, 'crossing_points': [0, ...",49.5,0.011685,20.283558,841.666667,1.033483,85.337649,99,2.801520,46.057539,0,-2.634892,0.629559


In [16]:
start = time.time()
features_tsfresh = extract_features(
    data, 
    column_id='id',   # Use the window ID as the column_id
    column_sort='time',      # Time column to sort within each window 
    default_fc_parameters=custom_fc_parameters, 
    n_jobs=1
)
end = time.time()

features_tsfresh.head()

Feature Extraction: 100%|██████████████████████████████████████████████████████████| 300/300 [00:00<00:00, 2188.37it/s]


,value1__number_crossing_m__m_0,value1__mean,value1__variance,value1__maximum,value1__minimum,value2__number_crossing_m__m_0,value2__mean,value2__variance,value2__maximum,value2__minimum,value3__number_crossing_m__m_0,value3__mean,value3__variance,value3__maximum,value3__minimum
0,1.0,49.5,833.25,99.0,0.0,45.0,0.179888,1.077621,3.186941,-2.245841,4.0,20.243567,95.094155,42.527250,-8.294389
1,1.0,49.5,833.25,99.0,0.0,46.0,-0.003050,1.019377,2.206501,-2.568951,2.0,21.497116,102.365797,51.297614,-0.267103
2,1.0,49.5,833.25,99.0,0.0,55.0,-0.016470,1.083712,1.887334,-2.979081,6.0,21.365738,101.045184,46.152546,-17.886584
3,1.0,49.5,833.25,99.0,0.0,55.0,-0.071130,0.880307,2.262206,-2.717986,6.0,20.507009,89.541889,46.448314,-8.567784
4,1.0,49.5,833.25,99.0,0.0,48.0,0.011685,1.023148,2.801520,-2.634892,0.0,20.283558,84.484273,46.057539,0.629559


In [64]:
records = []
names = []

rows, cols = features_it.shape

for i in range(0, cols // 3 - 1):
    record = []
    names.append(features_it.columns[i * 3 + 3])
    for j in range(0, rows):
        difference = features_it.iloc[j][i * 3 + 3] - features_tsfresh.iloc[j][i + 1]
        record.append(difference)
    records.append(record)

for i in range(0, cols // 3 - 1):
    record = []
    names.append(features_it.columns[i * 3 + 4])
    for j in range(0, rows):
        difference = features_it.iloc[j][i * 3 + 4] - features_tsfresh.iloc[j][i + 1 + 5]
        record.append(difference)
    records.append(record)

for i in range(0, cols // 3 - 1):
    record = []
    names.append(features_it.columns[i * 3 + 5])
    for j in range(0, rows):
        difference = features_it.iloc[j][i * 3 + 5] - features_tsfresh.iloc[j][i + 1 + 10]
        record.append(difference)
    records.append(record)

data = {}
for i in range(len(names)):
    data[names[i]] = records[i]

output_data = pd.DataFrame(
    
    data
)

output_data.to_csv('differences_output.csv', index=False, sep=';')

print("Differences written to 'differences_output.csv'")


C:\Users\jayke\AppData\Local\Temp\ipykernel_8516\3237416146.py:14: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  difference = features_it.iloc[j][i * 3 + 3] - features_tsfresh.iloc[j][i + 1]
C:\Users\jayke\AppData\Local\Temp\ipykernel_8516\3237416146.py:22: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  difference = features_it.iloc[j][i * 3 + 4] - features_tsfresh.iloc[j][i + 1 + 5]
C:\Users\jayke\AppData\Local\Temp\ipykernel_8516\3237416146.py:30: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame b

['mean_value1', 'variance_value1', 'peak_value1', 'trough_value1', 'mean_value2', 'variance_value2', 'peak_value2', 'trough_value2', 'mean_value3', 'variance_value3', 'peak_value3', 'trough_value3']
Differences written to 'differences_output.csv'
